In [21]:
import datetime
import itertools
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import random
import pickle
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
import torch
from tqdm import tqdm
from sklearn import preprocessing
# NLP
from sklearn.decomposition import PCA
import transformers
import ssl

In [9]:
df = pd.read_csv('/content/train.csv')

In [10]:
df.head()


,LOAN_ID,ORIGINAL_LANGUAGE,DESCRIPTION,DESCRIPTION_TRANSLATED,LOAN_AMOUNT,IMAGE_ID,ACTIVITY_NAME,SECTOR_NAME,LOAN_USE,COUNTRY_CODE,COUNTRY_NAME,TOWN_NAME,CURRENCY_POLICY,CURRENCY_EXCHANGE_COVERAGE_RATE,CURRENCY,TAGS,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL
0,1733169,English,Teodora is a 50-year-old married woman from th...,Teodora is a 50-year-old married woman from th...,100,3115271,Weaving,Arts,"to purchase materials like nipa palm, bamboo ...",PH,Philippines,"Maribojoc, Bohol",shared,0.1,PHP,#Elderly,monthly,field_partner
1,1546998,English,Diego is 32 years old and lives in the municip...,Diego is 32 years old and lives in the municip...,1350,2870403,Barber Shop,Services,"to buy two hair clippers, a new barber chair, ...",CO,Colombia,Apartadó,shared,0.1,COP,"user_favorite, user_favorite",monthly,field_partner
2,1808517,Spanish,"Osman, es un joven de 27 años de edad, soltero...","Osman is a young man, 27 years old, single, an...",225,3215705,Farming,Agriculture,to purchase sacks of fertilizers to care for a...,HN,Honduras,"Nueva Frontera, Santa Barbara.",shared,0.1,HNL,NaN,bullet,field_partner
3,1452940,English,"His name is Nino, 31 years old, married to Che...","His name is Nino, 31 years old, married to Che...",350,2745031,Motorcycle Transport,Transportation,"to pay for fuel, tires and change oil for his ...",PH,Philippines,"Silang, Cavite",shared,0.1,PHP,user_favorite,monthly,field_partner
4,1778420,English,"Pictured above is Teresa, often described as a...","Pictured above is Teresa, often described as a...",625,3083800,Farming,Agriculture,to purchase hybrid seeds and fertilizer to imp...,KE,Kenya,Mumias,shared,0.1,KES,"#Eco-friendly, #Sustainable Ag, #Parent, #Elde...",bullet,field_partner


In [11]:
description = df[['DESCRIPTION_TRANSLATED']]
description

,DESCRIPTION_TRANSLATED
0,Teodora is a 50-year-old married woman from th...
1,Diego is 32 years old and lives in the municip...
2,"Osman is a young man, 27 years old, single, an..."
3,"His name is Nino, 31 years old, married to Che..."
4,"Pictured above is Teresa, often described as a..."
...,...
91328,"Rider is 20 years old. He lives in San Javier,..."
91329,Carmelita works hard to support four children....
91330,"Orn, 60 years of age, appears in the photo. Sh..."
91331,"At 27 years of age, Walter is in a live-in rel..."


In [12]:
import re
def cleaning(text):
    text = re.sub("\n", " ", text) 
    text = re.sub("[^A-Za-z0-9]", " ", text) 
    text = re.sub("[' ']+", " ", text) 
    return text.lower() 

Creating a class for BERT 

In [13]:
import torch
import transformers

from transformers import BertTokenizer


class BertSequenceVectorizer:
    def __init__(self, model_name="bert-base-uncased", max_len=128):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_name = model_name
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = max_len

    def vectorize(self, sentence: str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy() # 0番目は [CLS] token, 768 dim の文章特徴量
        else:
            return seq_out[0][0].detach().numpy()

In [14]:
BSV = BertSequenceVectorizer(
    model_name="bert-base-uncased",
    max_len=128)
features = np.stack(
    description["DESCRIPTION_TRANSLATED"].fillna("").map(lambda x: BSV.vectorize(x).reshape(-1)).values
)
features

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will

array([[-0.24173544, -0.4984473 ,  0.5118569 , ..., -0.26480898,
         0.09317772,  0.08919012],
       [-0.21131033, -0.1476567 ,  0.5652367 , ..., -0.45297495,
         0.62749416,  0.16616036],
       [-0.2587864 ,  0.18665488,  0.26470667, ..., -0.42402166,
         0.33215922,  0.09634593],
       ...,
       [-0.12291255, -0.2524864 ,  0.04267163, ...,  0.04285635,
         0.2550213 , -0.03334789],
       [-0.12975581, -0.24003473,  0.39712143, ..., -0.15320235,
         0.17523946,  0.12909105],
       [ 0.08368339, -0.18113215, -0.04754119, ...,  0.06178788,
        -0.00276763,  0.7165465 ]], dtype=float32)

In [15]:
features.shape

(91333, 768)

In [17]:
train_description = pd.DataFrame(features)

Although the data tends to be huge and we do not want to use everything, we want to represent a multivariate data table as smaller set of variables (summary indices) in order to observe trends, jumps, clusters and outliers. In order to archive this, PCA will be implemented.

In [29]:
pca = PCA(n_components=0.90)
pca.fit_transform(train_description)
print(pca.explained_variance_ratio_)
print('Cumulative principal variance : ', np.sum(pca.explained_variance_ratio_)) 
print('The number of principal components: ', pca.n_components_) 

[0.39623847 0.05335096 0.04748863 0.02975008 0.0251465  0.02147628
 0.01842006 0.01704253 0.01324411 0.01205773 0.01134471 0.01039008
 0.00907982 0.00848714 0.00814909 0.00755839 0.00691977 0.0065606
 0.00638777 0.00576017 0.00560995 0.00523428 0.00495519 0.00491989
 0.0046147  0.00438272 0.00418341 0.004085   0.00390139 0.00379354
 0.00369959 0.00350213 0.00345559 0.00329167 0.00316319 0.00307948
 0.0030113  0.00282294 0.0027719  0.00271122 0.00262791 0.00254488
 0.00248884 0.00245975 0.00230217 0.00228757 0.00221996 0.0021925
 0.00208433 0.00200712 0.00197545 0.00193496 0.00186172 0.00184629
 0.00178695 0.00176984 0.00174845 0.00170443 0.00165341 0.00162624
 0.00159513 0.00157832 0.00154932 0.0015148  0.00149331 0.00147727
 0.00144697 0.00143882 0.00142343 0.00138243 0.00134838 0.00131378
 0.0012932  0.00127412 0.00124484 0.0012241  0.00120638 0.00119371
 0.00116081 0.00115036 0.00113948 0.00112076 0.00110393 0.0010806
 0.00106955 0.00106314 0.00104377 0.00103196 0.00101501 0.0010100

In [32]:
text_vec = pca.fit_transform(train_description)
output_df = pd.DataFrame(text_vec, columns=[f'bert_pca_vecs={i:03}' for i in range(text_vec.shape[1])])

In [34]:
output_df.head()

,bert_pca_vecs=000,bert_pca_vecs=001,bert_pca_vecs=002,bert_pca_vecs=003,bert_pca_vecs=004,bert_pca_vecs=005,bert_pca_vecs=006,bert_pca_vecs=007,bert_pca_vecs=008,bert_pca_vecs=009,bert_pca_vecs=010,bert_pca_vecs=011,bert_pca_vecs=012,bert_pca_vecs=013,bert_pca_vecs=014,bert_pca_vecs=015,bert_pca_vecs=016,bert_pca_vecs=017,bert_pca_vecs=018,bert_pca_vecs=019,bert_pca_vecs=020,bert_pca_vecs=021,bert_pca_vecs=022,bert_pca_vecs=023,bert_pca_vecs=024,bert_pca_vecs=025,bert_pca_vecs=026,bert_pca_vecs=027,bert_pca_vecs=028,bert_pca_vecs=029,bert_pca_vecs=030,bert_pca_vecs=031,bert_pca_vecs=032,bert_pca_vecs=033,bert_pca_vecs=034,bert_pca_vecs=035,bert_pca_vecs=036,bert_pca_vecs=037,bert_pca_vecs=038,bert_pca_vecs=039,...,bert_pca_vecs=086,bert_pca_vecs=087,bert_pca_vecs=088,bert_pca_vecs=089,bert_pca_vecs=090,bert_pca_vecs=091,bert_pca_vecs=092,bert_pca_vecs=093,bert_pca_vecs=094,bert_pca_vecs=095,bert_pca_vecs=096,bert_pca_vecs=097,bert_pca_vecs=098,bert_pca_vecs=099,bert_pca_vecs=100,bert_pca_vecs=101,bert_pca_vecs=102,bert_pca_vecs=103,bert_pca_vecs=104,bert_pca_vecs=105,bert_pca_vecs=106,bert_pca_vecs=107,bert_pca_vecs=108,bert_pca_vecs=109,bert_pca_vecs=110,bert_pca_vecs=111,bert_pca_vecs=112,bert_pca_vecs=113,bert_pca_vecs=114,bert_pca_vecs=115,bert_pca_vecs=116,bert_pca_vecs=117,bert_pca_vecs=118,bert_pca_vecs=119,bert_pca_vecs=120,bert_pca_vecs=121,bert_pca_vecs=122,bert_pca_vecs=123,bert_pca_vecs=124,bert_pca_vecs=125
0,-4.328352,0.546912,-0.890222,-0.937290,-0.763976,-0.342731,-0.074698,-0.079281,1.132151,-1.518477,-1.957097,1.230229,-0.513835,-0.968340,0.417453,-0.363493,-0.380266,0.622745,-0.195617,-0.764002,0.506976,-0.226333,0.094638,0.533665,0.283496,0.575223,-0.327142,-0.072799,-0.355666,-0.106975,0.797504,0.631558,-0.015597,0.264233,0.028232,0.541706,-0.290693,0.211101,-0.617502,0.390740,...,0.094950,-0.354929,-0.173599,-0.040717,-0.088743,-0.483733,0.049111,0.095800,0.281710,-0.081775,-0.129679,-0.464714,0.268350,0.297361,-0.163195,-0.561366,-0.025698,-0.316623,0.286750,-0.514209,0.398024,-0.012678,-0.018424,0.142255,-0.259022,0.013164,0.518170,-0.243524,0.051750,-0.054548,-0.106292,-0.217198,-0.525617,-0.148995,0.089628,-0.273710,0.137288,0.125735,0.095956,-0.073167
1,-4.699850,0.686877,1.969688,-0.876878,-0.373230,-1.609557,-1.547460,1.354966,-1.132529,0.330999,0.970820,0.514369,-0.415915,-0.706702,-0.957297,-0.066193,-0.168774,-0.024419,-0.071052,-0.496604,-0.079050,0.210807,-0.691994,-0.033140,-0.089081,-0.095193,0.146759,-0.266657,-0.162631,0.185473,0.505783,0.108108,-0.261950,-0.252206,0.581470,-0.053316,0.261154,0.119192,-0.073515,0.077395,...,-0.062307,0.300226,-0.037058,-0.533586,0.043165,0.059917,0.032858,0.083333,-0.270133,0.287607,-0.007174,0.224663,-0.009351,-0.105319,0.112879,0.192841,-0.036450,0.146479,-0.157552,-0.140528,-0.027969,0.183040,0.104634,0.072127,0.015056,0.003877,0.050074,-0.007436,-0.047990,-0.113402,-0.006318,-0.040764,-0.301444,-0.136051,-0.068044,0.156622,0.093983,-0.175176,0.216391,-0.309840
2,-5.302714,0.922105,2.197651,-0.869895,-0.820012,-0.744913,-1.939083,-0.029063,-0.064893,0.513086,-1.152323,0.407385,-0.222832,0.039545,0.942697,0.143969,0.106293,0.019555,-0.022076,-0.362315,-0.648385,-0.170088,0.235913,0.079518,-0.017074,-0.621319,-0.073742,-0.011357,0.189271,-0.011693,0.640814,-0.569899,0.143973,0.468014,0.380313,0.324636,-0.309307,0.164466,0.248885,0.008778,...,0.182805,0.248496,0.266619,-0.007481,0.244827,-0.158820,-0.132648,-0.356809,-0.153976,-0.344308,0.155031,-0.412190,0.069724,0.059806,0.062037,0.106321,0.150292,-0.221077,-0.541738,-0.040938,0.175859,-0.226996,0.507619,-0.166312,0.273928,0.008249,0.172922,0.320235,-0.180028,-0.117824,-0.116830,0.055781,-0.187977,-0.203321,-0.128703,0.155993,0.365985,0.104403,-0.102792,0.123171
3,-4.488940,0.777891,1.610945,-0.051437,-0.522079,-1.506257,-0.181531,0.485330,-0.365606,1.425000,-1.391174,0.410193,0.068125,-0.011631,-0.055643,0.406913,0.350087,-0.138597,-0.696896,-0.303579,-0.118768,0.045542,0.517630,-0.106194,-0.181882,0.2

Same Process for the test data

In [35]:
test = pd.read_csv('/content/sample_data/test.csv')

In [36]:
test_description = test[['DESCRIPTION_TRANSLATED']]

In [37]:
test_description

,DESCRIPTION_TRANSLATED
0,Marcela is 69 years old and married with ten c...
1,Roselia is 48 years old and has five children....
2,"Ma. Marebil is a single woman, 40 years old wi..."
3,"Good day, lenders! Meet one of KBMI’s clients,..."
4,Rosemarie is a married woman with two children...
...,...
91817,"Marjorie is a resident of Tubigon, Bohol. She ..."
91818,"Hello, Kiva community! Meet Janeth, a mother e..."
91819,Komi is 32 years old and married. He is a reno...
91820,"Kalbubu is 56 years old, a widow, and she has ..."


In [38]:
BSV = BertSequenceVectorizer(
    model_name="bert-base-uncased",
    max_len=128)
features_test = np.stack(
    test["DESCRIPTION_TRANSLATED"].fillna("").map(lambda x: BSV.vectorize(x).reshape(-1)).values
)
features_test

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will

array([[-0.02162604, -0.2814147 ,  0.13555394, ..., -0.33768883,
         0.11497159,  0.11687837],
       [ 0.05312145, -0.7417049 ,  0.21979214, ..., -0.59250534,
         0.1289405 ,  0.31543347],
       [ 0.06533924, -0.14293553, -0.149278  , ..., -0.27435064,
        -0.00088617,  0.07431131],
       ...,
       [-0.45251736,  0.05666868,  0.08293466, ..., -0.15872467,
         0.3543674 ,  0.39797008],
       [-0.11534022, -0.3343771 , -0.02148527, ...,  0.09149812,
         0.4145456 ,  0.3540789 ],
       [-0.06062814, -0.41311967, -0.1160955 , ..., -0.32696483,
         0.14316198, -0.03807072]], dtype=float32)

In [39]:
features_test.shape

(91822, 768)

In [40]:
test_description = pd.DataFrame(features_test)

In [41]:
test_text_vec = pca.transform(test_description)
output_test = pd.DataFrame(test_text_vec, columns=[f'bert_test_pca_vecs={i:03}' for i in range(test_text_vec.shape[1])])

In [42]:
output_test.head()

,bert_test_pca_vecs=000,bert_test_pca_vecs=001,bert_test_pca_vecs=002,bert_test_pca_vecs=003,bert_test_pca_vecs=004,bert_test_pca_vecs=005,bert_test_pca_vecs=006,bert_test_pca_vecs=007,bert_test_pca_vecs=008,bert_test_pca_vecs=009,bert_test_pca_vecs=010,bert_test_pca_vecs=011,bert_test_pca_vecs=012,bert_test_pca_vecs=013,bert_test_pca_vecs=014,bert_test_pca_vecs=015,bert_test_pca_vecs=016,bert_test_pca_vecs=017,bert_test_pca_vecs=018,bert_test_pca_vecs=019,bert_test_pca_vecs=020,bert_test_pca_vecs=021,bert_test_pca_vecs=022,bert_test_pca_vecs=023,bert_test_pca_vecs=024,bert_test_pca_vecs=025,bert_test_pca_vecs=026,bert_test_pca_vecs=027,bert_test_pca_vecs=028,bert_test_pca_vecs=029,bert_test_pca_vecs=030,bert_test_pca_vecs=031,bert_test_pca_vecs=032,bert_test_pca_vecs=033,bert_test_pca_vecs=034,bert_test_pca_vecs=035,bert_test_pca_vecs=036,bert_test_pca_vecs=037,bert_test_pca_vecs=038,bert_test_pca_vecs=039,...,bert_test_pca_vecs=086,bert_test_pca_vecs=087,bert_test_pca_vecs=088,bert_test_pca_vecs=089,bert_test_pca_vecs=090,bert_test_pca_vecs=091,bert_test_pca_vecs=092,bert_test_pca_vecs=093,bert_test_pca_vecs=094,bert_test_pca_vecs=095,bert_test_pca_vecs=096,bert_test_pca_vecs=097,bert_test_pca_vecs=098,bert_test_pca_vecs=099,bert_test_pca_vecs=100,bert_test_pca_vecs=101,bert_test_pca_vecs=102,bert_test_pca_vecs=103,bert_test_pca_vecs=104,bert_test_pca_vecs=105,bert_test_pca_vecs=106,bert_test_pca_vecs=107,bert_test_pca_vecs=108,bert_test_pca_vecs=109,bert_test_pca_vecs=110,bert_test_pca_vecs=111,bert_test_pca_vecs=112,bert_test_pca_vecs=113,bert_test_pca_vecs=114,bert_test_pca_vecs=115,bert_test_pca_vecs=116,bert_test_pca_vecs=117,bert_test_pca_vecs=118,bert_test_pca_vecs=119,bert_test_pca_vecs=120,bert_test_pca_vecs=121,bert_test_pca_vecs=122,bert_test_pca_vecs=123,bert_test_pca_vecs=124,bert_test_pca_vecs=125
0,6.170295,-0.690748,-1.865736,-0.306904,-1.595511,-1.846105,0.367540,0.081063,-0.683653,0.127085,-0.407418,0.929732,0.182640,0.100178,-0.185200,0.255256,-0.318823,-0.228494,-0.157352,0.033441,0.155045,0.169257,-0.500950,0.024516,-0.518127,-0.296274,0.061615,0.135870,0.155312,0.027033,-0.295363,-0.346842,0.218114,-0.035163,-0.049701,0.278694,0.136370,0.396159,0.221567,-0.014382,...,0.122885,-0.256893,-0.047725,-0.103842,0.012098,-0.032087,-0.098145,-0.156835,0.028237,0.135659,0.010488,0.081279,0.193733,0.090977,-0.129076,-0.078388,-0.094722,0.140208,0.022556,-0.170197,0.084198,-0.054360,0.059121,0.116398,-0.147553,-0.045018,-0.067477,-0.189828,0.019575,-0.125346,-0.166338,-0.083075,0.110702,0.089197,-0.033235,-0.168182,-0.156713,0.027880,0.047643,0.090934
1,-3.255446,-0.283549,-3.525486,0.052102,-0.278628,-1.646948,-0.388735,0.560824,-0.107734,-1.864557,-0.712423,0.601644,1.197472,0.353958,0.572607,1.211849,-0.624038,-0.194473,0.638242,0.002410,0.843246,0.856276,0.076612,-0.207161,0.542033,0.394053,-0.044305,-0.023075,0.151455,0.155093,-0.489161,-0.052052,-0.502948,0.246793,-0.547196,0.136511,0.346534,0.501351,-0.006699,-0.397846,...,0.284042,0.034535,-0.014942,0.085014,-0.209729,0.211264,0.194088,0.162280,-0.285862,-0.480975,-0.018913,0.108930,-0.073131,-0.201755,-0.290395,0.406593,-0.061091,-0.075158,-0.210524,-0.102651,-0.041063,-0.420346,-0.098285,-0.203286,0.174093,-0.254699,0.194612,-0.244954,0.318366,0.112201,-0.018799,-0.009158,-0.101865,0.212605,-0.033933,0.077178,-0.345054,0.349607,0.171169,-0.040666
2,6.610266,-0.797629,-0.687071,1.126617,-1.068634,-0.582964,0.707720,-0.266406,0.292716,-0.336308,-0.356667,0.327231,-0.598059,0.903781,-0.569790,0.449259,0.972390,0.211276,0.425722,0.941577,-0.865074,0.791385,0.690819,0.960800,-0.028790,-0.017075,0.341140,0.100032,-0.153660,0.242144,0.150386,0.065117,0.516765,0.103205,-0.082316,0.371630,-0.234286,-0.223301,0.322273,0.294250,...,0.084699,-0.124412,-0.157243,0.050559,-0.142554,0.142227,0.213074,0.007635,0.057468,-0.168902,0.129415,-0.044885,-0.257105,0.072263,-0.035209,-0.117190,-0.461318,-0.353407,-0.098215,0.286757,-0.134173,0.076995,0.113942,0.195168,0.228033,

In [44]:
output_df.to_csv('translated_df.csv',index=False)
output_test.to_csv('translated_test.csv',index=False)